In [1]:
#importing keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM, Input, SpatialDropout1D
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# DataFrame
import pandas as pd
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
#numpy
import numpy as np
#utility
import re

In [3]:
# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [4]:
#Load Dataset
df = pd.read_csv("~/library/sentiment_data.csv")

In [5]:
vf = pd.read_csv("~/library/Validation_data_competition.csv")

In [6]:
#Load Dataset
tf = pd.read_csv("~/library/testset-competition.csv",)

In [7]:
tokenizer = Tokenizer(num_words=None, split=' ')
tokenizer.fit_on_texts(df['tweet_content'].values)
x = tokenizer.texts_to_sequences(df['tweet_content'].values)
x = pad_sequences(x, maxlen=20, padding='post')
print(x[54])

[   6 4324  367    5   92    8   79    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [8]:
y = tokenizer.texts_to_sequences(vf['tweet_content'].values)
y = pad_sequences(y, maxlen=20, padding='post')
print(y[3])

[225  16 873 310 165 420 281   1   1   4   1 836   1  44   1 923   1   3
   0   0]


In [9]:
from sklearn.model_selection import train_test_split
x_test = df.iloc[:,4:].values
y_test=vf.iloc[:,3:].values

In [10]:
batch_size = 16
epochs = 5

In [11]:
model = Sequential()
model.add(Embedding(14645, 256))
model.add(Dropout(0.4))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.1))
model.add(Dense(4, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 256)         3749120   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                36992     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total params: 3,786,244
Trainable params: 3,786,244
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
history=model.fit(x, x_test,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(y, y_test))

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Train on 9087 samples, validate on 2125 samples
Epoch 1/5
9087/9087 [==============================] - 60s - loss: 0.2708 - acc: 0.8861 - val_loss: 0.1855 - val_acc: 0.9486
Epoch 2/5
9087/9087 [==============================] - 56s - loss: 0.1479 - acc: 0.9551 - val_loss: 0.1916 - val_acc: 0.9415
Epoch 3/5
9087/9087 [==============================] - 58s - loss: 0.0988 - acc: 0.9701 - val_loss: 0.2318 - val_acc: 0.9309
Epoch 4/5
9087/9087 [==============================] - 63s - loss: 0.0664 - acc: 0.9787 - val_loss: 0.2750 - val_acc: 0.9171
Epoch 5/5
8256/9087 [==========================>...] - ETA: 5s - loss: 0.0461 - acc: 0.9845

In [14]:
score= model.evaluate(y, y_test, verbose = 2)
print("Loss: %.2f" % score[0])
print("Accuracy: %.2f" % score[1])

Loss: 0.20
Accuracy: 0.94


In [162]:
prediction = model.predict(x)

In [163]:
print('Prediction value:', prediction[374])

Prediction value: [0.9517962  0.44485503 0.45648408 0.19614866]


In [147]:
z = tokenizer.texts_to_sequences(tf['tweet_content'].values)
z = pad_sequences(z, maxlen=22, padding='post')

In [182]:
pred=model.predict(z)
print('Prediction value:', pred[420])

Prediction value: [0.6747369  0.19792125 0.34736517 0.02598023]


In [184]:
for i in range(len(pred)):
    tf.at[i,'Harassment']=int(0)
    tf.at[i,'IndirectH']=int(0)
    tf.at[i,'PhysicalH']=int(0)
    tf.at[i,'SexualH']=int(0)
    if pred[i,0]>=0.4:
        tf.at[i,'Harassment']=1
        t=np.argmax(pred[i,1:])
        if t==2:
            tf.at[i,'SexualH']=int(1)
        elif t==1:
            tf.at[i,'PhysicalH']=int(1)
        elif t==0:
            tf.at[i,'IndirectH']=int(1)

In [185]:
tf[420:431]

,tweet_content,Harassment,IndirectH,PhysicalH,SexualH
420,naked girls fingering eachother beautiful girl...,1.0,0.0,1.0,0.0
421,naked girls vigina sexy swedish men pics,1.0,0.0,0.0,1.0
422,naked pillow fights nude girls in the farm,0.0,0.0,0.0,0.0
423,nanuqsaurus was a tyrannosaur from the late cr...,0.0,0.0,0.0,0.0
424,nasa commanded voyager to fire its thrusters f...,0.0,0.0,0.0,0.0
425,nature at its best in troms norway by tommy si...,0.0,0.0,0.0,0.0
426,nawww im the last person to criticize other pp...,0.0,0.0,0.0,0.0
427,nba young boy has a shitty ass personality his...,1.0,0.0,1.0,0.0
428,nba youngboy beat his gf ass blacked her eye a...,1.0,0.0,0.0,1.0
429,need a bitch who know algebra,1.0,0.0,0.0,1.0


In [49]:
model.save('model(2).h5')

In [186]:
tf.to_csv('~/library/testset-competition-result(8).csv')

In [140]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')